# DDU Documents Merger

이 노트북은 두 개의 DDU (Document Decomposition Unit) pickle 파일을 하나로 병합합니다.

## 병합할 파일:
1. `gv80_owners_manual_TEST6P_documents.pkl` - GV80 차량 매뉴얼
2. `디지털정부혁신_추진계획_documents.pkl` - 디지털 정부 혁신 계획 문서

## DDU Document Schema
각 문서는 Langchain Document 객체로 다음 구조를 가집니다:
- `page_content`: 실제 텍스트 내용
- `metadata`: 14개 카테고리와 다양한 메타데이터

## 1. 필요한 라이브러리 임포트

In [6]:
import pickle
import os
from pathlib import Path
from typing import List, Dict, Any
import json
from datetime import datetime

# Langchain Document 타입 임포트
try:
    from langchain.schema import Document
except ImportError:
    from langchain_core.documents import Document

print(f"작업 디렉토리: {os.getcwd()}")
print(f"실행 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

작업 디렉토리: /mnt/e/MyProject2/multimodal-rag-wsl-v2
실행 시간: 2025-08-26 16:10:23


## 2. 데이터 파일 경로 설정

In [7]:
# 데이터 디렉토리 경로
DATA_DIR = Path("data")

# 입력 파일 경로
GV80_FILE = DATA_DIR / "gv80_owners_manual_TEST6P_documents.pkl"
DIGITAL_GOV_FILE = DATA_DIR / "디지털정부혁신_추진계획_documents.pkl"

# 출력 파일 경로
MERGED_FILE = DATA_DIR / "merged_ddu_documents.pkl"

# 파일 존재 확인
print(f"GV80 파일 존재: {GV80_FILE.exists()}")
print(f"디지털정부 파일 존재: {DIGITAL_GOV_FILE.exists()}")

GV80 파일 존재: True
디지털정부 파일 존재: True


## 3. Pickle 파일 로드 및 구조 확인

In [8]:
def load_pickle_file(file_path: Path) -> List[Document]:
    """Pickle 파일을 로드하고 기본 정보를 출력합니다."""
    print(f"\n로딩 중: {file_path.name}")
    
    with open(file_path, 'rb') as f:
        documents = pickle.load(f)
    
    print(f"- 문서 개수: {len(documents)}")
    
    # 카테고리 분포 확인
    categories = {}
    for doc in documents:
        category = doc.metadata.get('category', 'unknown')
        categories[category] = categories.get(category, 0) + 1
    
    print("- 카테고리 분포:")
    for cat, count in sorted(categories.items()):
        print(f"  • {cat}: {count}")
    
    # 첫 번째 문서 샘플 확인
    if documents:
        first_doc = documents[0]
        print(f"\n- 첫 번째 문서 메타데이터 키: {list(first_doc.metadata.keys())}")
        print(f"- 소스: {first_doc.metadata.get('source', 'N/A')}")
    
    return documents

# 두 파일 로드
gv80_docs = load_pickle_file(GV80_FILE)
digital_gov_docs = load_pickle_file(DIGITAL_GOV_FILE)


로딩 중: gv80_owners_manual_TEST6P_documents.pkl
- 문서 개수: 122
- 카테고리 분포:
  • caption: 4
  • figure: 10
  • header: 3
  • heading1: 23
  • list: 1
  • paragraph: 78
  • table: 3

- 첫 번째 문서 메타데이터 키: ['source', 'page', 'category', 'id', 'raw_output', 'translation_text', 'translation_markdown', 'translation_html', 'contextualize_text', 'caption', 'entity', 'image_path', 'coordinates', 'processing_type', 'processing_status', 'source_parser', 'element_type', 'human_feedback']
- 소스: data/gv80_owners_manual_TEST6P.pdf

로딩 중: 디지털정부혁신_추진계획_documents.pkl
- 문서 개수: 158
- 카테고리 분포:
  • figure: 3
  • header: 2
  • heading1: 34
  • list: 4
  • paragraph: 104
  • table: 11

- 첫 번째 문서 메타데이터 키: ['source', 'page', 'category', 'id', 'raw_output', 'translation_text', 'translation_markdown', 'translation_html', 'contextualize_text', 'caption', 'entity', 'image_path', 'coordinates', 'processing_type', 'processing_status', 'source_parser', 'element_type', 'human_feedback']
- 소스: data/디지털정부혁신_추진계획.pdf


## 4. 문서 병합 전 검증

병합 전에 확인해야 할 사항:
- ID 중복 여부
- 메타데이터 구조 일관성
- 소스 경로 충돌

In [9]:
def validate_documents(docs1: List[Document], docs2: List[Document]) -> Dict[str, Any]:
    """두 문서 세트의 호환성을 검증합니다."""
    validation_result = {
        'id_conflicts': [],
        'metadata_keys_diff': set(),
        'source_files': set(),
        'total_pages': set(),
        'categories': set()
    }
    
    # ID 수집
    ids1 = {doc.metadata.get('id') for doc in docs1 if doc.metadata.get('id')}
    ids2 = {doc.metadata.get('id') for doc in docs2 if doc.metadata.get('id')}
    
    # ID 충돌 확인
    validation_result['id_conflicts'] = list(ids1.intersection(ids2))
    
    # 메타데이터 키 비교
    keys1 = set(docs1[0].metadata.keys()) if docs1 else set()
    keys2 = set(docs2[0].metadata.keys()) if docs2 else set()
    validation_result['metadata_keys_diff'] = keys1.symmetric_difference(keys2)
    
    # 소스 파일 수집
    for doc in docs1 + docs2:
        if 'source' in doc.metadata:
            validation_result['source_files'].add(doc.metadata['source'])
        if 'page' in doc.metadata:
            validation_result['total_pages'].add(doc.metadata['page'])
        if 'category' in doc.metadata:
            validation_result['categories'].add(doc.metadata['category'])
    
    return validation_result

# 검증 실행
validation = validate_documents(gv80_docs, digital_gov_docs)

print("\n=== 병합 전 검증 결과 ===")
print(f"ID 충돌 개수: {len(validation['id_conflicts'])}")
if validation['id_conflicts']:
    print(f"  충돌 ID: {validation['id_conflicts'][:5]}...")

print(f"\n메타데이터 키 차이: {validation['metadata_keys_diff'] if validation['metadata_keys_diff'] else '없음'}")
print(f"\n고유 소스 파일: {len(validation['source_files'])}개")
for source in validation['source_files']:
    print(f"  - {source}")

print(f"\n전체 카테고리 종류: {len(validation['categories'])}개")
print(f"  {', '.join(sorted(validation['categories']))}")


=== 병합 전 검증 결과 ===
ID 충돌 개수: 116
  충돌 ID: [1, 2, 3, 4, 5]...

메타데이터 키 차이: 없음

고유 소스 파일: 2개
  - data/gv80_owners_manual_TEST6P.pdf
  - data/디지털정부혁신_추진계획.pdf

전체 카테고리 종류: 7개
  caption, figure, header, heading1, list, paragraph, table


## 5. 문서 병합 로직

병합 시 고려사항:
- ID 충돌 시 소스별 prefix 추가
- 메타데이터 무결성 유지
- 원본 순서 보존

In [11]:
def merge_documents(
    docs1: List[Document], 
    docs2: List[Document],
    reorganize_ids: bool = True,
    include_category_in_id: bool = False
) -> tuple[List[Document], Dict[str, str]]:
    """두 문서 리스트를 병합하고 새로운 ID 체계를 적용합니다.
    
    Args:
        docs1: 첫 번째 문서 리스트
        docs2: 두 번째 문서 리스트
        reorganize_ids: 모든 문서에 새 ID 부여 여부
        include_category_in_id: ID에 카테고리 포함 여부
    
    Returns:
        tuple: (병합된 문서 리스트, ID 매핑 딕셔너리)
    """
    import copy
    import re
    from collections import defaultdict
    
    merged_docs = []
    id_mapping = {}  # old_id -> new_id 매핑
    
    def extract_source_prefix(source_path: str) -> str:
        """소스 파일명에서 의미있는 prefix를 추출합니다.
        파일명을 그대로 사용하되, 확장자와 특수문자는 제거합니다.
        """
        import os
        
        # 파일명만 추출 (경로 제거)
        filename = os.path.basename(source_path)
        
        # 확장자 제거
        name_without_ext = os.path.splitext(filename)[0]
        
        # 특수문자를 언더스코어로 변환하고 소문자로 변환
        # 한글은 유지하되, 공백과 특수문자는 언더스코어로 변환
        clean_name = re.sub(r'[^\w가-힣]+', '_', name_without_ext)
        clean_name = clean_name.strip('_')  # 앞뒤 언더스코어 제거
        
        # 너무 길면 적절히 자름 (최대 20자)
        # if len(clean_name) > 20:
        #     clean_name = clean_name[:20]
        
        return clean_name.lower()
    
    def get_category_abbr(category: str) -> str:
        """카테고리를 짧은 약어로 변환합니다."""
        category_map = {
            'heading1': 'h1',
            'heading2': 'h2', 
            'heading3': 'h3',
            'paragraph': 'para',
            'list': 'list',
            'table': 'tbl',
            'figure': 'fig',
            'chart': 'chrt',
            'equation': 'eq',
            'caption': 'cap',
            'footnote': 'fn',
            'header': 'hdr',
            'footer': 'ftr',
            'reference': 'ref'
        }
        return category_map.get(category, 'unk')
    
    def process_documents(docs: List[Document], default_prefix: str) -> List[Document]:
        """문서 리스트를 처리하고 새 ID를 부여합니다."""
        processed_docs = []
        
        # source별로 문서 그룹화
        docs_by_source = defaultdict(list)
        for doc in docs:
            source = doc.metadata.get('source', 'unknown')
            docs_by_source[source].append(doc)
        
        # 각 source별로 처리
        for source, source_docs in docs_by_source.items():
            prefix = extract_source_prefix(source) if source != 'unknown' else default_prefix
            
            # 카테고리별로 카운터 관리 (옵션)
            if include_category_in_id:
                category_counters = defaultdict(int)
                
                for doc in source_docs:
                    new_doc = copy.deepcopy(doc)
                    category = doc.metadata.get('category', 'unknown')
                    cat_abbr = get_category_abbr(category)
                    
                    category_counters[cat_abbr] += 1
                    new_id = f"{prefix}_{cat_abbr}_{category_counters[cat_abbr]:03d}"
                    
                    # 원본 ID 보존
                    if 'id' in new_doc.metadata:
                        old_id = new_doc.metadata['id']
                        new_doc.metadata['original_id'] = old_id
                        id_mapping[old_id] = new_id
                    
                    new_doc.metadata['id'] = new_id
                    new_doc.metadata['id_source_prefix'] = prefix
                    processed_docs.append(new_doc)
            else:
                # 단순 순차 번호
                for idx, doc in enumerate(source_docs, 1):
                    new_doc = copy.deepcopy(doc)
                    new_id = f"{prefix}_{idx:04d}"
                    
                    # 원본 ID 보존
                    if 'id' in new_doc.metadata:
                        old_id = new_doc.metadata['id']
                        new_doc.metadata['original_id'] = old_id
                        id_mapping[old_id] = new_id
                    
                    new_doc.metadata['id'] = new_id
                    new_doc.metadata['id_source_prefix'] = prefix
                    processed_docs.append(new_doc)
        
        return processed_docs
    
    # 두 문서 세트 처리
    print("📝 ID 재구성 중...")
    processed_docs1 = process_documents(docs1, 'doc1')
    processed_docs2 = process_documents(docs2, 'doc2')
    
    merged_docs = processed_docs1 + processed_docs2
    
    # ID 재구성 통계 출력
    print(f"\n✅ ID 재구성 완료!")
    print(f"  - 총 {len(id_mapping)}개 ID 매핑 생성")
    
    # prefix별 통계
    prefix_stats = defaultdict(int)
    for doc in merged_docs:
        prefix = doc.metadata.get('id_source_prefix', 'unknown')
        prefix_stats[prefix] += 1
    
    print("\n📊 Prefix별 문서 수:")
    for prefix, count in sorted(prefix_stats.items()):
        print(f"  - {prefix}: {count}개")
    
    # 중복 ID 체크
    all_new_ids = [doc.metadata.get('id') for doc in merged_docs if 'id' in doc.metadata]
    unique_ids = set(all_new_ids)
    if len(all_new_ids) != len(unique_ids):
        print(f"\n⚠️  경고: {len(all_new_ids) - len(unique_ids)}개의 중복 ID 발견!")
    else:
        print(f"\n✅ 모든 ID가 고유합니다.")
    
    return merged_docs, id_mapping

# 병합 실행 (카테고리 포함 옵션)
print("옵션 1: 카테고리를 포함한 ID 생성")
merged_documents_with_cat, id_mapping_with_cat = merge_documents(
    gv80_docs, 
    digital_gov_docs, 
    include_category_in_id=True
)

print("\n" + "="*50)
print("\n옵션 2: 단순 순차 ID 생성")
merged_documents, id_mapping = merge_documents(
    gv80_docs, 
    digital_gov_docs, 
    include_category_in_id=False
)

print(f"\n병합 완료!")
print(f"총 문서 수: {len(merged_documents)}")
print(f"예상 문서 수: {len(gv80_docs) + len(digital_gov_docs)}")
print(f"일치 여부: {len(merged_documents) == len(gv80_docs) + len(digital_gov_docs)}")

# 샘플 ID 확인
print("\n📋 새 ID 샘플 (처음 5개):")
for doc in merged_documents[:5]:
    old_id = doc.metadata.get('original_id', 'N/A')
    new_id = doc.metadata.get('id', 'N/A')
    category = doc.metadata.get('category', 'N/A')
    source_prefix = doc.metadata.get('id_source_prefix', 'N/A')
    print(f"  [{source_prefix}] {old_id} → {new_id} (카테고리: {category})")

옵션 1: 카테고리를 포함한 ID 생성
📝 ID 재구성 중...

✅ ID 재구성 완료!
  - 총 163개 ID 매핑 생성

📊 Prefix별 문서 수:
  - gv80_owners_manual_test6p: 122개
  - 디지털정부혁신_추진계획: 158개

✅ 모든 ID가 고유합니다.


옵션 2: 단순 순차 ID 생성
📝 ID 재구성 중...

✅ ID 재구성 완료!
  - 총 163개 ID 매핑 생성

📊 Prefix별 문서 수:
  - gv80_owners_manual_test6p: 122개
  - 디지털정부혁신_추진계획: 158개

✅ 모든 ID가 고유합니다.

병합 완료!
총 문서 수: 280
예상 문서 수: 280
일치 여부: True

📋 새 ID 샘플 (처음 5개):
  [gv80_owners_manual_test6p] 0 → gv80_owners_manual_test6p_0001 (카테고리: heading1)
  [gv80_owners_manual_test6p] 1 → gv80_owners_manual_test6p_0002 (카테고리: heading1)
  [gv80_owners_manual_test6p] 2 → gv80_owners_manual_test6p_0003 (카테고리: figure)
  [gv80_owners_manual_test6p] 3 → gv80_owners_manual_test6p_0004 (카테고리: table)
  [gv80_owners_manual_test6p] 4 → gv80_owners_manual_test6p_0005 (카테고리: caption)


## 6. 병합 결과 검증

In [12]:
def analyze_merged_documents(documents: List[Document]) -> None:
    """병합된 문서의 통계를 분석합니다."""
    
    # 소스별 분류
    sources = {}
    for doc in documents:
        source = doc.metadata.get('source', 'unknown')
        if source not in sources:
            sources[source] = []
        sources[source].append(doc)
    
    print("\n=== 병합 결과 분석 ===")
    print(f"\n소스별 문서 분포:")
    for source, docs in sources.items():
        print(f"  {source}: {len(docs)} 문서")
    
    # 카테고리별 통계
    categories_by_source = {}
    for doc in documents:
        source = doc.metadata.get('source', 'unknown')
        category = doc.metadata.get('category', 'unknown')
        
        if source not in categories_by_source:
            categories_by_source[source] = {}
        
        if category not in categories_by_source[source]:
            categories_by_source[source][category] = 0
        
        categories_by_source[source][category] += 1
    
    print("\n소스별 카테고리 분포:")
    for source, categories in categories_by_source.items():
        print(f"\n  [{source}]")
        for cat, count in sorted(categories.items()):
            print(f"    • {cat}: {count}")
    
    # 메타데이터 완전성 체크
    required_fields = ['source', 'category', 'id', 'page']
    incomplete_docs = []
    
    for i, doc in enumerate(documents):
        missing = [field for field in required_fields if field not in doc.metadata]
        if missing:
            incomplete_docs.append((i, missing))
    
    if incomplete_docs:
        print(f"\n⚠️  메타데이터 불완전 문서: {len(incomplete_docs)}개")
        for idx, missing in incomplete_docs[:5]:
            print(f"    문서 {idx}: {missing} 누락")
    else:
        print(f"\n✅ 모든 문서의 메타데이터가 완전합니다.")

# 분석 실행
analyze_merged_documents(merged_documents)


=== 병합 결과 분석 ===

소스별 문서 분포:
  data/gv80_owners_manual_TEST6P.pdf: 122 문서
  data/디지털정부혁신_추진계획.pdf: 158 문서

소스별 카테고리 분포:

  [data/gv80_owners_manual_TEST6P.pdf]
    • caption: 4
    • figure: 10
    • header: 3
    • heading1: 23
    • list: 1
    • paragraph: 78
    • table: 3

  [data/디지털정부혁신_추진계획.pdf]
    • figure: 3
    • header: 2
    • heading1: 34
    • list: 4
    • paragraph: 104
    • table: 11

✅ 모든 문서의 메타데이터가 완전합니다.


## 7. 병합된 문서 저장

In [13]:
def save_merged_documents(
    documents: List[Document], 
    output_path: Path,
    create_backup: bool = True
) -> None:
    """병합된 문서를 pickle 파일로 저장합니다."""
    
    # 백업 생성 (기존 파일이 있는 경우)
    if create_backup and output_path.exists():
        backup_path = output_path.with_suffix(f'.backup_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pkl')
        print(f"기존 파일 백업: {backup_path}")
        output_path.rename(backup_path)
    
    # 저장
    with open(output_path, 'wb') as f:
        pickle.dump(documents, f)
    
    file_size = output_path.stat().st_size / (1024 * 1024)  # MB로 변환
    print(f"\n✅ 저장 완료!")
    print(f"  파일: {output_path}")
    print(f"  크기: {file_size:.2f} MB")
    print(f"  문서 수: {len(documents)}")
    
    # 메타데이터 파일도 생성 (JSON)
    metadata_path = output_path.with_suffix('.metadata.json')
    metadata = {
        'created_at': datetime.now().isoformat(),
        'total_documents': len(documents),
        'sources': list({doc.metadata.get('source') for doc in documents if 'source' in doc.metadata}),
        'categories': list({doc.metadata.get('category') for doc in documents if 'category' in doc.metadata}),
        'merged_from': [
            str(GV80_FILE),
            str(DIGITAL_GOV_FILE)
        ]
    }
    
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)
    
    print(f"  메타데이터: {metadata_path}")

# 저장 실행
save_merged_documents(merged_documents, MERGED_FILE)


✅ 저장 완료!
  파일: data/merged_ddu_documents.pkl
  크기: 0.84 MB
  문서 수: 280
  메타데이터: data/merged_ddu_documents.metadata.json


In [14]:
def save_id_mapping(id_mapping: Dict[str, str], output_path: Path = None) -> None:
    """ID 매핑을 JSON 파일로 저장합니다."""
    if output_path is None:
        output_path = DATA_DIR / "id_mapping.json"
    
    # ID 매핑 정보 확장
    mapping_info = {
        'created_at': datetime.now().isoformat(),
        'total_mappings': len(id_mapping),
        'mapping': id_mapping,
        'statistics': {}
    }
    
    # prefix별 통계 계산
    prefix_counts = {}
    for new_id in id_mapping.values():
        prefix = new_id.split('_')[0]
        prefix_counts[prefix] = prefix_counts.get(prefix, 0) + 1
    
    mapping_info['statistics']['by_prefix'] = prefix_counts
    
    # JSON으로 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(mapping_info, f, ensure_ascii=False, indent=2)
    
    print(f"📁 ID 매핑 저장: {output_path}")
    print(f"   - 총 {len(id_mapping)}개 매핑")
    for prefix, count in prefix_counts.items():
        print(f"   - {prefix}: {count}개")

# ID 매핑 저장
save_id_mapping(id_mapping)

# ========================================================================
# ID 패턴 분석 로직 - 완전 구현
# ========================================================================

def analyze_id_patterns(documents: List[Document], id_mapping: Dict[str, str]) -> Dict[str, Any]:
    """ID 패턴을 상세하게 분석합니다."""
    analysis_result = {
        'total_documents': len(documents),
        'id_duplicates': [],
        'category_distribution': {},
        'page_distribution': {},
        'prefix_patterns': {},
        'consistency_check': {},
        'id_length_stats': {}
    }
    
    # 1. ID 중복 체크
    all_ids = []
    id_counter = {}
    for doc in documents:
        doc_id = doc.metadata.get('id')
        if doc_id:
            all_ids.append(doc_id)
            id_counter[doc_id] = id_counter.get(doc_id, 0) + 1
    
    # 중복 ID 찾기
    duplicates = {id: count for id, count in id_counter.items() if count > 1}
    analysis_result['id_duplicates'] = duplicates
    
    # 2. 카테고리별 ID 분포
    category_ids = {}
    for doc in documents:
        category = doc.metadata.get('category', 'unknown')
        doc_id = doc.metadata.get('id', 'no_id')
        prefix = doc.metadata.get('id_source_prefix', 'unknown')
        
        if category not in category_ids:
            category_ids[category] = {'count': 0, 'prefixes': set(), 'sample_ids': []}
        
        category_ids[category]['count'] += 1
        category_ids[category]['prefixes'].add(prefix)
        if len(category_ids[category]['sample_ids']) < 3:
            category_ids[category]['sample_ids'].append(doc_id)
    
    # Set을 리스트로 변환
    for cat in category_ids:
        category_ids[cat]['prefixes'] = list(category_ids[cat]['prefixes'])
    
    analysis_result['category_distribution'] = category_ids
    
    # 3. 페이지별 ID 순서 분석
    page_ids = {}
    for doc in documents:
        page = doc.metadata.get('page', 'unknown')
        doc_id = doc.metadata.get('id', 'no_id')
        source = doc.metadata.get('source', 'unknown')
        
        page_key = f"{source}:p{page}"
        if page_key not in page_ids:
            page_ids[page_key] = []
        page_ids[page_key].append(doc_id)
    
    # 각 페이지의 ID 수와 순서 정보 저장
    for page_key in page_ids:
        page_ids[page_key] = {
            'count': len(page_ids[page_key]),
            'ids': page_ids[page_key][:5],  # 처음 5개만 샘플로
            'is_sequential': check_sequential(page_ids[page_key])
        }
    
    analysis_result['page_distribution'] = page_ids
    
    # 4. Prefix 패턴 분석
    prefix_patterns = {}
    for doc in documents:
        prefix = doc.metadata.get('id_source_prefix', 'unknown')
        doc_id = doc.metadata.get('id', 'no_id')
        
        if prefix not in prefix_patterns:
            prefix_patterns[prefix] = {
                'count': 0,
                'id_format_samples': set(),
                'categories': set()
            }
        
        prefix_patterns[prefix]['count'] += 1
        prefix_patterns[prefix]['categories'].add(doc.metadata.get('category', 'unknown'))
        
        # ID 형식 샘플 저장 (처음 5개만)
        if len(prefix_patterns[prefix]['id_format_samples']) < 5:
            prefix_patterns[prefix]['id_format_samples'].add(doc_id)
    
    # Set을 리스트로 변환
    for prefix in prefix_patterns:
        prefix_patterns[prefix]['id_format_samples'] = list(prefix_patterns[prefix]['id_format_samples'])
        prefix_patterns[prefix]['categories'] = list(prefix_patterns[prefix]['categories'])
    
    analysis_result['prefix_patterns'] = prefix_patterns
    
    # 5. ID 길이 통계
    id_lengths = [len(doc_id) for doc_id in all_ids if doc_id]
    if id_lengths:
        analysis_result['id_length_stats'] = {
            'min': min(id_lengths),
            'max': max(id_lengths),
            'avg': sum(id_lengths) / len(id_lengths),
            'most_common': max(set(id_lengths), key=id_lengths.count)
        }
    
    return analysis_result

def check_sequential(ids: List[str]) -> bool:
    """ID 리스트가 순차적인지 확인합니다."""
    try:
        # ID에서 숫자 부분만 추출
        numbers = []
        for id in ids:
            # 마지막 언더스코어 이후의 숫자 추출
            parts = id.split('_')
            if parts:
                last_part = parts[-1]
                if last_part.isdigit():
                    numbers.append(int(last_part))
        
        # 순차적인지 확인
        if numbers:
            sorted_nums = sorted(numbers)
            for i in range(1, len(sorted_nums)):
                if sorted_nums[i] - sorted_nums[i-1] != 1:
                    return False
            return True
    except:
        pass
    
    return False

# ID 패턴 분석 실행
print("\n" + "="*70)
print("🔍 ID 패턴 상세 분석")
print("="*70)

analysis = analyze_id_patterns(merged_documents, id_mapping)

# 1. 중복 ID 체크
print("\n📌 ID 중복 체크:")
if analysis['id_duplicates']:
    print(f"  ⚠️  {len(analysis['id_duplicates'])}개의 중복 ID 발견!")
    for dup_id, count in list(analysis['id_duplicates'].items())[:5]:
        print(f"    - {dup_id}: {count}회 중복")
else:
    print("  ✅ 모든 ID가 고유합니다.")

# 2. 카테고리별 분포
print("\n📊 카테고리별 ID 분포:")
print(f"  {'카테고리':<15} | {'문서수':<8} | {'사용 Prefix':<30} | {'샘플 ID'}")
print("  " + "-"*90)
for category, info in sorted(analysis['category_distribution'].items(), 
                            key=lambda x: x[1]['count'], reverse=True):
    prefixes = ', '.join(info['prefixes'][:3])
    sample_id = info['sample_ids'][0] if info['sample_ids'] else 'N/A'
    print(f"  {category:<15} | {info['count']:<8} | {prefixes:<30} | {sample_id}")

# 3. 페이지별 분석 (상위 5개만)
print("\n📄 페이지별 ID 순서 분석 (샘플):")
page_items = sorted(analysis['page_distribution'].items(), 
                   key=lambda x: x[1]['count'], reverse=True)[:5]
for page_key, info in page_items:
    seq_status = "순차적" if info['is_sequential'] else "비순차적"
    print(f"  {page_key}: {info['count']}개 문서 ({seq_status})")
    if info['ids']:
        print(f"    샘플: {', '.join(info['ids'][:3])}")

# 4. Prefix 패턴 분석
print("\n🏷️  Prefix 패턴 분석:")
for prefix, pattern in sorted(analysis['prefix_patterns'].items(), 
                              key=lambda x: x[1]['count'], reverse=True):
    print(f"\n  [{prefix}] - {pattern['count']}개 문서")
    print(f"    카테고리: {', '.join(pattern['categories'][:5])}")
    print(f"    ID 샘플: {', '.join(pattern['id_format_samples'][:3])}")

# 5. ID 길이 통계
if analysis['id_length_stats']:
    stats = analysis['id_length_stats']
    print("\n📏 ID 길이 통계:")
    print(f"  최소: {stats['min']}자")
    print(f"  최대: {stats['max']}자")
    print(f"  평균: {stats['avg']:.1f}자")
    print(f"  가장 흔한 길이: {stats['most_common']}자")

# 6. ID 매핑 변환 예시
print("\n🔄 ID 변환 예시 (10개):")
print(f"  {'Original ID':<45} → {'New ID':<25}")
print("  " + "-"*75)
for i, (old_id, new_id) in enumerate(list(id_mapping.items())[:10]):
    truncated_old = old_id[:45] if len(old_id) > 45 else old_id
    print(f"  {truncated_old:<45} → {new_id:<25}")

# 분석 결과를 JSON으로도 저장
analysis_path = DATA_DIR / "id_analysis_report.json"
with open(analysis_path, 'w', encoding='utf-8') as f:
    # numpy/datetime 객체를 JSON 직렬화 가능하도록 변환
    json_safe_analysis = json.loads(json.dumps(analysis, default=str))
    json.dump(json_safe_analysis, f, ensure_ascii=False, indent=2)

print(f"\n📊 분석 보고서 저장: {analysis_path}")

📁 ID 매핑 저장: data/id_mapping.json
   - 총 163개 매핑
   - 디지털정부혁신: 158개
   - gv80: 5개

🔍 ID 패턴 상세 분석

📌 ID 중복 체크:
  ✅ 모든 ID가 고유합니다.

📊 카테고리별 ID 분포:
  카테고리            | 문서수      | 사용 Prefix                      | 샘플 ID
  ------------------------------------------------------------------------------------------
  paragraph       | 182      | 디지털정부혁신_추진계획, gv80_owners_manual_test6p | gv80_owners_manual_test6p_0006
  heading1        | 57       | 디지털정부혁신_추진계획, gv80_owners_manual_test6p | gv80_owners_manual_test6p_0001
  table           | 14       | 디지털정부혁신_추진계획, gv80_owners_manual_test6p | gv80_owners_manual_test6p_0004
  figure          | 13       | 디지털정부혁신_추진계획, gv80_owners_manual_test6p | gv80_owners_manual_test6p_0003
  header          | 5        | 디지털정부혁신_추진계획, gv80_owners_manual_test6p | gv80_owners_manual_test6p_0044
  list            | 5        | 디지털정부혁신_추진계획, gv80_owners_manual_test6p | gv80_owners_manual_test6p_0095
  caption         | 4        | gv80_owners_manual_test6p      | gv80_o

TypeError: object of type 'int' has no len()

def save_merged_documents(
    documents: List[Document], 
    output_path: Path,
    id_mapping: Dict[str, str] = None,
    create_backup: bool = True
) -> None:
    """병합된 문서를 pickle 파일로 저장합니다."""
    
    # 백업 생성 (기존 파일이 있는 경우)
    if create_backup and output_path.exists():
        backup_path = output_path.with_suffix(f'.backup_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pkl')
        print(f"기존 파일 백업: {backup_path}")
        output_path.rename(backup_path)
    
    # 저장
    with open(output_path, 'wb') as f:
        pickle.dump(documents, f)
    
    file_size = output_path.stat().st_size / (1024 * 1024)  # MB로 변환
    print(f"\n✅ 저장 완료!")
    print(f"  파일: {output_path}")
    print(f"  크기: {file_size:.2f} MB")
    print(f"  문서 수: {len(documents)}")
    
    # 메타데이터 파일도 생성 (JSON)
    metadata_path = output_path.with_suffix('.metadata.json')
    
    # ID 체계 정보 수집
    id_prefixes = set()
    categories_used = set()
    sources_used = set()
    
    for doc in documents:
        if 'id_source_prefix' in doc.metadata:
            id_prefixes.add(doc.metadata['id_source_prefix'])
        if 'category' in doc.metadata:
            categories_used.add(doc.metadata['category'])
        if 'source' in doc.metadata:
            sources_used.add(doc.metadata['source'])
    
    metadata = {
        'created_at': datetime.now().isoformat(),
        'total_documents': len(documents),
        'sources': list(sources_used),
        'categories': list(categories_used),
        'id_prefixes': list(id_prefixes),
        'id_reorganized': id_mapping is not None,
        'total_id_mappings': len(id_mapping) if id_mapping else 0,
        'merged_from': [
            str(GV80_FILE),
            str(DIGITAL_GOV_FILE)
        ]
    }
    
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)
    
    print(f"  메타데이터: {metadata_path}")
    
    # ID 매핑도 저장 (있는 경우)
    if id_mapping:
        mapping_path = output_path.with_suffix('.id_mapping.json')
        with open(mapping_path, 'w', encoding='utf-8') as f:
            json.dump({
                'created_at': datetime.now().isoformat(),
                'total_mappings': len(id_mapping),
                'mapping': id_mapping
            }, f, ensure_ascii=False, indent=2)
        print(f"  ID 매핑: {mapping_path}")

# 저장 실행
save_merged_documents(merged_documents, MERGED_FILE, id_mapping)

## 8. 검증: 저장된 파일 다시 로드

In [15]:
# 저장된 파일을 다시 로드하여 무결성 확인
print("저장된 파일 검증 중...")

with open(MERGED_FILE, 'rb') as f:
    loaded_docs = pickle.load(f)

print(f"\n✅ 검증 결과:")
print(f"  로드된 문서 수: {len(loaded_docs)}")
print(f"  원본 문서 수: {len(merged_documents)}")
print(f"  일치: {len(loaded_docs) == len(merged_documents)}")

# 데이터 무결성 상세 검증
def verify_data_integrity(original_docs: List[Document], loaded_docs: List[Document]) -> Dict[str, Any]:
    """저장/로드 후 데이터 무결성을 검증합니다."""
    integrity_report = {
        'document_count_match': len(original_docs) == len(loaded_docs),
        'id_match': True,
        'metadata_match': True,
        'content_match': True,
        'errors': []
    }
    
    # 문서별 검증
    for i, (orig, loaded) in enumerate(zip(original_docs, loaded_docs)):
        # ID 검증
        if orig.metadata.get('id') != loaded.metadata.get('id'):
            integrity_report['id_match'] = False
            integrity_report['errors'].append(f"Document {i}: ID mismatch")
        
        # 메타데이터 키 검증
        if set(orig.metadata.keys()) != set(loaded.metadata.keys()):
            integrity_report['metadata_match'] = False
            integrity_report['errors'].append(f"Document {i}: Metadata keys mismatch")
        
        # 컨텐츠 검증
        if orig.page_content != loaded.page_content:
            integrity_report['content_match'] = False
            integrity_report['errors'].append(f"Document {i}: Content mismatch")
    
    return integrity_report

# 무결성 검증 실행
integrity = verify_data_integrity(merged_documents, loaded_docs)

print("\n🔒 데이터 무결성 검증:")
print(f"  문서 수 일치: {'✅' if integrity['document_count_match'] else '❌'}")
print(f"  ID 일치: {'✅' if integrity['id_match'] else '❌'}")
print(f"  메타데이터 일치: {'✅' if integrity['metadata_match'] else '❌'}")
print(f"  컨텐츠 일치: {'✅' if integrity['content_match'] else '❌'}")

if integrity['errors']:
    print(f"\n  ⚠️  발견된 오류:")
    for error in integrity['errors'][:5]:
        print(f"    - {error}")
else:
    print("\n  ✨ 완벽한 데이터 무결성!")

# 샘플 문서 상세 확인
if loaded_docs:
    print("\n📋 샘플 문서 상세 정보:")
    for i in range(min(3, len(loaded_docs))):
        doc = loaded_docs[i]
        print(f"\n  문서 #{i+1}:")
        print(f"    ID: {doc.metadata.get('id', 'N/A')}")
        print(f"    원본 ID: {doc.metadata.get('original_id', 'N/A')}")
        print(f"    Prefix: {doc.metadata.get('id_source_prefix', 'N/A')}")
        print(f"    카테고리: {doc.metadata.get('category', 'N/A')}")
        print(f"    소스: {doc.metadata.get('source', 'N/A')}")
        print(f"    페이지: {doc.metadata.get('page', 'N/A')}")
        print(f"    내용 길이: {len(doc.page_content)} 글자")
        print(f"    내용 미리보기: {doc.page_content[:50]}...")

저장된 파일 검증 중...

✅ 검증 결과:
  로드된 문서 수: 280
  원본 문서 수: 280
  일치: True

🔒 데이터 무결성 검증:
  문서 수 일치: ✅
  ID 일치: ✅
  메타데이터 일치: ✅
  컨텐츠 일치: ✅

  ✨ 완벽한 데이터 무결성!

📋 샘플 문서 상세 정보:

  문서 #1:
    ID: gv80_owners_manual_test6p_0001
    원본 ID: 0
    Prefix: gv80_owners_manual_test6p
    카테고리: heading1
    소스: data/gv80_owners_manual_TEST6P.pdf
    페이지: 1
    내용 길이: 17 글자
    내용 미리보기: # 내 용 찾 기 방 법 설 명...

  문서 #2:
    ID: gv80_owners_manual_test6p_0002
    원본 ID: 1
    Prefix: gv80_owners_manual_test6p
    카테고리: heading1
    소스: data/gv80_owners_manual_TEST6P.pdf
    페이지: 1
    내용 길이: 11 글자
    내용 미리보기: # 내용으로 찾을 때...

  문서 #3:
    ID: gv80_owners_manual_test6p_0003
    원본 ID: 2
    Prefix: gv80_owners_manual_test6p
    카테고리: figure
    소스: data/gv80_owners_manual_TEST6P.pdf
    페이지: 1
    내용 길이: 1 글자
    내용 미리보기: 
...


## 9. 요약 및 다음 단계

### 완료된 작업:
- ✅ 두 개의 DDU pickle 파일 로드
- ✅ 문서 구조 및 호환성 검증
- ✅ ID 충돌 처리 및 병합
- ✅ 병합된 문서 저장 및 메타데이터 생성

### 다음 단계 제안:
1. **데이터베이스 인제스트**: 병합된 문서를 PostgreSQL에 인제스트
2. **임베딩 생성**: 새로운 문서들에 대한 벡터 임베딩 생성
3. **검색 테스트**: 병합된 데이터셋으로 검색 성능 테스트

In [16]:
print("\n" + "="*70)
print("🎉 병합 작업 완료!")
print("="*70)

# 최종 요약 통계
from datetime import datetime

print("\n📊 최종 통계:")
print(f"  작업 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"  총 병합 문서: {len(merged_documents)}개")
print(f"    - GV80 문서: {len(gv80_docs)}개")
print(f"    - 디지털정부 문서: {len(digital_gov_docs)}개")

# Prefix별 최종 통계
prefix_counts = {}
for doc in merged_documents:
    prefix = doc.metadata.get('id_source_prefix', 'unknown')
    prefix_counts[prefix] = prefix_counts.get(prefix, 0) + 1

print(f"\n  생성된 Prefix:")
for prefix, count in sorted(prefix_counts.items()):
    print(f"    - {prefix}: {count}개")

# 카테고리 요약
category_counts = {}
for doc in merged_documents:
    category = doc.metadata.get('category', 'unknown')
    category_counts[category] = category_counts.get(category, 0) + 1

print(f"\n  카테고리 분포:")
top_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)[:5]
for category, count in top_categories:
    percentage = (count / len(merged_documents)) * 100
    print(f"    - {category}: {count}개 ({percentage:.1f}%)")

# 생성된 파일 목록
print("\n📁 생성된 파일:")
output_files = [
    (MERGED_FILE, "병합된 DDU 문서"),
    (MERGED_FILE.with_suffix('.metadata.json'), "메타데이터"),
    (MERGED_FILE.with_suffix('.id_mapping.json'), "ID 매핑 테이블"),
    (DATA_DIR / "id_analysis_report.json", "ID 분석 보고서")
]

for file_path, description in output_files:
    if file_path.exists():
        file_size = file_path.stat().st_size
        if file_size > 1024*1024:
            size_str = f"{file_size/(1024*1024):.2f} MB"
        elif file_size > 1024:
            size_str = f"{file_size/1024:.2f} KB"
        else:
            size_str = f"{file_size} bytes"
        print(f"  ✅ {file_path.name} - {description} ({size_str})")
    else:
        print(f"  ❌ {file_path.name} - 파일 없음")

print("\n🚀 다음 단계:")
print("1. 데이터베이스 인제스트:")
print(f"   python scripts/2_phase1_ingest_documents.py --pickle-file {MERGED_FILE}")
print("")
print("2. 임베딩 생성 및 벡터 저장:")
print("   - 한국어/영어 이중 언어 임베딩 생성")
print("   - PostgreSQL + pgvector에 저장")
print("")
print("3. 검색 테스트:")
print("   python scripts/test_retrieval_real_data.py")
print("")
print("4. 워크플로우 실행:")
print("   python scripts/test_workflow_complete.py")

print("\n💡 팁:")
print("  - ID 매핑 파일을 활용하여 원본 ID 추적 가능")
print("  - 메타데이터 파일로 데이터 구조 빠른 파악 가능")
print("  - ID 분석 보고서로 데이터 품질 모니터링 가능")

print("\n" + "="*70)
print("모든 작업이 성공적으로 완료되었습니다! 🎊")
print("="*70)


🎉 병합 작업 완료!

📊 최종 통계:
  작업 시간: 2025-08-26 22:19:10
  총 병합 문서: 280개
    - GV80 문서: 122개
    - 디지털정부 문서: 158개

  생성된 Prefix:
    - gv80_owners_manual_test6p: 122개
    - 디지털정부혁신_추진계획: 158개

  카테고리 분포:
    - paragraph: 182개 (65.0%)
    - heading1: 57개 (20.4%)
    - table: 14개 (5.0%)
    - figure: 13개 (4.6%)
    - header: 5개 (1.8%)

📁 생성된 파일:
  ✅ merged_ddu_documents.pkl - 병합된 DDU 문서 (860.35 KB)
  ✅ merged_ddu_documents.metadata.json - 메타데이터 (445 bytes)
  ❌ merged_ddu_documents.id_mapping.json - 파일 없음
  ❌ id_analysis_report.json - 파일 없음

🚀 다음 단계:
1. 데이터베이스 인제스트:
   python scripts/2_phase1_ingest_documents.py --pickle-file data/merged_ddu_documents.pkl

2. 임베딩 생성 및 벡터 저장:
   - 한국어/영어 이중 언어 임베딩 생성
   - PostgreSQL + pgvector에 저장

3. 검색 테스트:
   python scripts/test_retrieval_real_data.py

4. 워크플로우 실행:
   python scripts/test_workflow_complete.py

💡 팁:
  - ID 매핑 파일을 활용하여 원본 ID 추적 가능
  - 메타데이터 파일로 데이터 구조 빠른 파악 가능
  - ID 분석 보고서로 데이터 품질 모니터링 가능

모든 작업이 성공적으로 완료되었습니다! 🎊
